In [1]:
# imports de bibliotecas
import os
import numpy as np

In [ ]:
ARQUIVO_ANALEX = 'entrada.txt'
TABELA_SIMBOLOS = 'tabela.txt'

ABRE_PARENTESES = 0
FECHA_PARENTESES = 1
RAIZ = 2

SOMA = 0
SUBTRACAO = 1
MULTIPLICACAO = 2
DIVISAO = 3
POTENCIA = 4

SINAL = 0
NUMERO = 1
PALAVRA_RES = 2

# Declaração de códigos de erro
ARQUIVO_INVALIDO = 1
CARACTERE_INVALIDO = 2
# Declaração de todos os sinais e palavras reservadas da linguagem
sinais = ['+', '-', '*', '/', '^']
palavras_reservadas = ['(', ')', '√']
tipo_token = ['SINAL', 'NUMERO', 'PALAVRA_RESERVADA']

parser_table = []

def carregarArquivo(arquivo):
  lista_diretorio = os.listdir()
  if arquivo in lista_diretorio:
    entrada = open(arquivo, 'r')
    return entrada.read()
  else:
      return False

def tratarErros(erro):
  if erro == ARQUIVO_INVALIDO:
    print('Arquivo não encontrado.')
  elif erro == CARACTERE_INVALIDO:
    print('Caractere inválido encontrado.')

def salvarToken(palavra, classificacao):
  parser_table.append([tipo_token[classificacao], palavra])

def imprimirTabela():
  with open(TABELA_SIMBOLOS, 'w') as filehandle:
    for i in parser_table:
      filehandle.write(str(i[0]) + ',' + str(i[1]) + '\n')
  for item in parser_table:
    print(item[0], item[1])

def carregarTabela(arquivo):
  token = []
  lista_tokens = []
  tabela_carregada = open(arquivo, 'r')
  linhas = len(tabela_carregada.readlines())
  tabela_carregada.seek(0)
  for i in range(linhas):
    token.append(tabela_carregada.readline())
    token[i] = token[i].rstrip("\n")
    lista_tokens.append(token[i].split(',', 1))
  return lista_tokens

def analex(nome_arquivo):
  arquivo = carregarArquivo(nome_arquivo)
  erro = False
  if arquivo:
    print('Arquivo carregado!\n')
  else:
    tratarErros(ARQUIVO_INVALIDO)
    return
  palavra = ''
  classificacao = -1
  for index, letra in enumerate(arquivo):
    if letra.isdigit():
      palavra += letra
      classificacao = NUMERO
      if index == len(arquivo)-1:
        salvarToken(palavra, classificacao)
    elif letra in sinais:
      if palavra:
        salvarToken(palavra, classificacao)
        palavra = ''
      classificacao = SINAL
      salvarToken(letra, classificacao)
    elif letra in palavras_reservadas:
      if palavra:
        salvarToken(palavra, classificacao)
        palavra = ''
      classificacao = PALAVRA_RES
      salvarToken(letra, classificacao)
    else:
      erro = True
      tratarErros(CARACTERE_INVALIDO)
      break
  if not erro:
    imprimirTabela()

def anasint(tabela):
  controleParenteses = 0
  if tabela[0][1] != palavras_reservadas[ABRE_PARENTESES] and not tabela[0][1].isdigit() and tabela[0][1] != palavras_reservadas[RAIZ]:
      print('A expressão deve iniciar com abre parênteses, raiz ou um número.')
  for i in range(len(tabela)):
    if tabela[i][1] == palavras_reservadas[ABRE_PARENTESES]:
      controleParenteses += 1
      if not i == len(tabela)-1:
        if tabela[i+1][1] == sinais[MULTIPLICACAO] or tabela[i+1][1] == sinais[DIVISAO] or tabela[i+1][1] == sinais[POTENCIA]:
          print("Só é permitido operador de soma, subtração ou raiz depois do abre paretenses.")
        if tabela[i+1][1] == palavras_reservadas[FECHA_PARENTESES]:
          print("Não é permitido fechar o parentese sem indicar uma operação de valores!")
        if i > 1:
          if not (tabela[i-1][1] in sinais or tabela[i-1][1] == palavras_reservadas[ABRE_PARENTESES]):
            print("É preciso vir um sinal antes de uma operação com parenteses")
      else:
        print("Um parêntese não pode ser aberto no fim da expressão.")
    if tabela[i][1] == palavras_reservadas[FECHA_PARENTESES]:
      controleParenteses -= 1
      if controleParenteses < 0:
        print("A ordem dos parenteses esta incorreta.")
      if not i == len(tabela)-1:
        if tabela[i+1][1] == palavras_reservadas[RAIZ]:
          print("Um sinal deve preceder a raiz quadrada quando estiver depois de um fecha parentese.")
    if tabela[i][1] == palavras_reservadas[RAIZ]:
      if i == len(tabela)-1:
        print("Um número deve vir após a raiz.")
      elif not i == len(tabela)-1:
        if not tabela[i+1][1].isdigit():
          print("Um número deve vir após a raiz.")
    if tabela[i][1] == sinais[POTENCIA]:
      if not (tabela[i-1][1].isdigit() or tabela[i-1][1] == palavras_reservadas[FECHA_PARENTESES]):
        print("Um número ou fecha parentese deve vir antes do sinal de potencia.")
      if i == len(tabela)-1:
        print("Um número deve vir após a potencia.")
      elif not i == len(tabela)-1:
        if not (tabela[i+1][1].isdigit() or tabela[i+1][1] == palavras_reservadas[ABRE_PARENTESES] or tabela[i+1][1] == sinais[SUBTRACAO]):
          print("Um número deve vir após a potencia.")
    if tabela[i][1] == sinais[SUBTRACAO] or tabela[i][1] == sinais[SOMA] or tabela[i][1] == sinais[MULTIPLICACAO] or tabela[i][1] == sinais[DIVISAO]:
      if i == len(tabela)-1:
        print("Um número deve vir após um sinal de operação.")
      elif not i == len(tabela)-1:
        if not (tabela[i+1][1].isdigit() or tabela[i+1][1] == palavras_reservadas[ABRE_PARENTESES]):
          print("Um número deve vir após um sinal de operação.")
    if i == len(tabela)-1 and controleParenteses > 0:
      print("A expressão ainda contém parenteses abertos.")
    if i == len(tabela)-1 and controleParenteses < 0:
      print("Não é permitido fechar o parentese sem abri-lo anteriormente.")
    
analex(ARQUIVO_ANALEX)
print('\n')
anasint(carregarTabela(TABELA_SIMBOLOS))
